In [1]:
#Train-Test-Split. Split data into training and testing sets, with "isFraud" as the target variable
y=cleaned_df["isFraud"]
X=cleaned_df.drop("isFraud", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

NameError: name 'cleaned_df' is not defined

In [3]:
# Log Transformation to skewed numerical data
X_train["amount"] = np.log1p(X_train["amount"])
X_test["amount"] = np.log1p(X_test["amount"])

In [4]:
# Create binary features by using one-hot encoding to convert categorical variables to binary
#X_train = pd.get_dummies(X_train, columns=["type"], drop_first=True)
#X_test = pd.get_dummies(X_test, columns=["type"],drop_first=True)

In [5]:
# Logistic Regression Model
#logreg = LogisticRegression(max_iter=1000)
#logreg.fit(X_train, y_train)

#y_probs = logreg.predict_proba(X_test)[:, 1]

In [6]:
# ROC Curve
#fpr, tpr, thresholds = roc_curve(y_test, y_probs)
#roc_auc = auc(fpr, tpr)

#plt.figure(figsize=(8,6))
#plt.plot(fpr, tpr, color="darkorange", lw=2, label="ROC Curve (area= %0.2f)" % roc_auc)
#plt.plot([0,1], [0,1], color="navy", lw=2, linestyle='--')
#plt.xlim([0.0, 1.0])
#plt.ylim([0.0, 1.05])
#plt.xlabel("False Positive Rate")
#plt.ylabel("True Positive Rate")
#plt.title("ROC Curve")
#plt.legend(loc="lower right")
#plt.show()

In [7]:
# Support Vector Machine (SVM) Classifier
#svm = SVC(kernel="linear")
#svm.fit(X_train, y_train)

#y_pred = svm.predict(X_test)

#classification_report(y_test, y_pred)